In [0]:
processName = dbutils.widgets.get('prm_processName')
 
nextSourceFileDataSql = f""" 
    select nvl(max (processed_file_table_date)+1,'2023-01-01') as next_source_file_date 
    from pricing_analytics_vrs.processrunlogs.deltalakehouse_process_runs
    where process_name = '{processName}' 
    and process_status = 'completed' 
"""

nextSourceFileDataDF = spark.sql(nextSourceFileDataSql)



In [0]:
from datetime import datetime

In [0]:
dailyPricingSourceBaseURL = 'https://retailpricing.blob.core.windows.net/'
dailyPricingSourceFolder = 'daily-pricing/'
daiilyPricingSourceFileDate = datetime.strptime(str(nextSourceFileDataDF.select('next_source_file_date').collect()[0]['next_source_file_date']),'%Y-%m-%d').strftime('%d%m%Y')
daiilyPricingSourceFileName = f"PW_MW_DR_{daiilyPricingSourceFileDate}.csv"

daiilyPricingSinkLayerName = 'bronze'
daiilyPricingSinkStorageAccountName = 'azuredatalakevrs'
daiilyPricingSinkFolderName =  'daily-pricing'


In [0]:
%python
import pandas as pds

In [0]:
dailyPricingSourceURL = dailyPricingSourceBaseURL + dailyPricingSourceFolder + daiilyPricingSourceFileName


In [0]:
dailyPricingPandasDF = pds.read_csv(dailyPricingSourceURL)



In [0]:
dailyPricingSparkDF= spark.createDataFrame(dailyPricingPandasDF)


In [0]:
from pyspark.sql.functions import current_timestamp
dailyPricingSinkFolderPath = f"abfss://{daiilyPricingSinkLayerName}@{daiilyPricingSinkStorageAccountName}.dfs.core.windows.net/{daiilyPricingSinkFolderName}"


(
    dailyPricingSparkDF
    .withColumn("source_file_load_date",current_timestamp())
    .write
    .mode("append")
    .option("header", "true")
    .csv(dailyPricingSinkFolderPath)
)

In [0]:

processFileDate = nextSourceFileDataDF.select('next_source_file_date').collect()[0]['next_source_file_date']
processStatus = 'completed'

processInsertSql = f""" insert into pricing_analytics_vrs.processrunlogs.Deltalakehouse_Process_Runs 
(process_name,processed_file_table_date,process_status)
values ('{processName}','{processFileDate}','{processStatus}')"""

spark.sql(processInsertSql)